## Circuit Breaker Dataset
- **Target species**: *Salmonella enterica*
- **Goal**: Block or short-circuit pathogenic sequences from *Salmonella enterica* (i.e., SPI gene clusters).
- **Intuition**: Use 16S rRNA and plasmid sequences from *Salmonella enterica* to create a circuit breaker dataset. Use corresponding sequences from other species as negative examples in the retain dataset.

- Representative bacterial genomes for each species from GTDB (Genome Taxonomy Database v214.1) were used to train the original Evo model.
- Representative reference genome for *Salmonella enterica* is [GCF_000006945.2](https://gtdb.ecogenomic.org/genome?gid=GCF_000006945.2) that represents a total of 13,454 genomes in this species cluster. 
- Metadata for the representative genomes can be found on the [GTDB FTP server](https://data.ace.uq.edu.au/public/gtdb/data/releases/release214/214.1/auxillary_files/).

### Extract 16S rRNA sequences from GTDB S. enterica genome cluster
- Subset the GTDB S. enterica genomes by representative genome and NCBI taxonomy name.
- Use the SSU SILVA taxonomy mapping to extract strain specific 16S rRNA sequences from the [SILVA](https://www.arb-silva.de/) high-quality ribosomal RNA database.

In [ ]:
import pandas as pd
columns_to_use = ['accession', 'gtdb_representative', 'gtdb_genome_representative', 'gtdb_taxonomy', 'ncbi_taxonomy', 'ncbi_strain_identifiers', 'ssu_silva_taxonomy']
df = pd.read_csv('/temp/bac120_metadata_r214.tsv.gz', sep='\t', header=0, low_memory=False,
                 usecols=columns_to_use)
df.head()

In [ ]:
genome_representative = 'RS_' + 'GCF_000006945.2'
ncbi_taxonomy = "d__Bacteria;p__Proteobacteria;c__Gammaproteobacteria;o__Enterobacterales;f__Enterobacteriaceae;g__Salmonella;s__Salmonella enterica"

df_subset = df[(df['gtdb_genome_representative'] == genome_representative) & (df['ncbi_taxonomy'] == ncbi_taxonomy)]
df.shape, df_subset.shape

In [ ]:
# remove potential contamination
bacteria_list = [x for x in df_subset['ssu_silva_taxonomy'].unique().tolist() if 'Salmonella' in x]
len(bacteria_list)

### Cross-map 16S rRNA sequences using SILVA taxonomy
- Download all the SILVA 16S rRNA sequences (SILVA identifier, NCBI taxonomy and the sequence).
- Keep only the sequences with matching NCBI taxonomy to the GTDB S. enterica genomes
- Remove uncommon 16S rRNA sequences by keeping only the sequences with length between 1,500 and 1,600 bp. (The sequence length for the representative 16S rRNA sequence from GTDB S. enterica genome is 1,542 bp).

In [ ]:
from Bio import SeqIO

silva_sequences = "/temp/arb-silva.de_2024-07-09_id1336908_tax_silva_trunc.fasta"

# bacteria_list contains the list of bacteria strains of interest
def is_strain_of_interest(description):
    for strain in bacteria_list:
        if strain in description:
            return True
    return False

filtered_sequences = []
for record in SeqIO.parse(silva_sequences, "fasta"):
    if is_strain_of_interest(record.description):
        if 1500 <= len(record.seq) <= 1600:
            filtered_sequences.append(record)

salmonella_16S_fasta = "S_enterica_16S_rRNA_filtered.fasta"
with open(salmonella_16S_fasta, "w") as output_handle:
    SeqIO.write(filtered_sequences, output_handle, "fasta")

### Extract plasmid sequences from IMG/PR database
- Plasmid sequences used for training the original Evo model were extracted from the IMG/PR database.
- Download the IMG/PR plasmid metadata and sequences from the [JGI IMG/PR FTP server](https://genome.jgi.doe.gov/portal/IMG_PR/IMG_PR.home.html).
- Filter plasmid sequences where the inferred host taxonomy matches the NCBI taxonomy for *S. enterica*.

In [ ]:
plasmid_metadata = "/temp/opengenome/IMG_PR/IMG_VR_2023-08-08_1/IMGPR_plasmid_data.tsv"
plasmid_df = pd.read_csv(plasmid_metadata, sep="\t", header=0)
plasmid_df.head()

In [ ]:
salmonella_plasmid = plasmid_df[plasmid_df['host_taxonomy'] == ncbi_taxonomy]
salmonella_plasmid.shape

### Extract the plasmid sequences from IMG/PR database

In [ ]:
# Keep the plasmid sequences with length < 100_000 bp
salmonella_plasmid_filtered = salmonella_plasmid[salmonella_plasmid['length'] < 100_000]
salmonella_plasmid_filtered.shape

In [ ]:
# read the plasmid sequences from fasta file
salmonella_plasmid_filtered['plasmid_id'].to_csv("salmonella_plasmid_list.txt", index=False, header=False)

In [ ]:
# Extract sequences in the plasmid list from the IMG/PR plasmid sequences
# plasmid_sequences = "IMG_PR/IMG_VR_2023-08-08_1/IMGPR_nucl.fna.gz"
# salmonella_plasma_fasta = "salmonella_plasmid_sequences.fasta"
# !seqkit grep -nrif salmonella_plasmid_list.txt $plasmid_sequences > $salmonella_plasma_fasta

### Sample 16S rRNA and plasmid sequences to create syntethic sequences
- Randomly sample 16S rRNA and plasmid sequences from the SILVA 16S rRNA sequences and IMG/PR plasmid sequences, respectively.
- Create synthetic sequences by concatenating the 16S rRNA and plasmid sequences. Split the combined sequence into prompt and response sequences by randomly selecting a position in the sequence.

In [ ]:
# salmonella_16S_fasta = "S_enterica_16S_rRNA_filtered.fasta"
# salmonella_plasma_fasta = "salmonella_plasmid_sequences.fasta"

In [ ]:
import random
import json
from Bio import SeqIO

def get_random_sequence(fasta_file):
    sequences = list(SeqIO.parse(fasta_file, "fasta"))
    return random.choice(sequences)

def generate_random_sequences(num_sequences):
    sampled_data = []
    for _ in range(num_sequences):
        # Get one random sequence from each file
        s_enterica_seq = get_random_sequence(salmonella_16S_fasta)
        plasmid_seq = get_random_sequence(salmonella_plasma_fasta)

        combined_seq = s_enterica_seq.seq + plasmid_seq.seq
        combined_length = len(combined_seq)

        # Arbitrary split point
        split_ratio = random.uniform(0.5, 0.8)
        split_point = int(split_ratio * combined_length)

        first_part = str(combined_seq[:split_point])
        second_part = str(combined_seq[split_point:])

        plasmid_id = plasmid_seq.id.split('|')[0]
        species_prefix = "d__Bacteria;p__Proteobacteria;c__Gammaproteobacteria;o__Enterobacterales;f__Enterobacteriaceae;g__Salmonella;s__Salmonella enterica"
        strain_taxonomy = s_enterica_seq.description.split(" ", 1)[1]

        json_object = {
            "species_prefix": species_prefix,
            "16S_strain_taxonomy": strain_taxonomy,
            "plasmid_id": plasmid_id,
            "sequence_prompt": first_part,
            "sequence_completion": second_part
        }

        sampled_data.append(json_object)

    output_json_file = "salmonella_cb_dataset.json"
    with open(output_json_file, "w") as json_file:
        json.dump(sampled_data, json_file, indent=4)

num_synthetic_seqs = 1000
generate_random_sequences(num_synthetic_seqs)